# Aula 3 - Tratamento de Dados Categóricos

Na aula de hoje, vamos conversar um pouco sobre:

- 1. Codificação de variáveis categóricas
- 2. Label Encoding
- 3. OneHot Encoding
- 4. ColumnTransformer e FunctionTransformer
- 5. Target Encoder

## 1. Codificação de Variáveis Categóricas

A grande maioria dos modelos Machine Learning são implementações de equações matemáticas, teoremas e relações de álgebra linear (lembrem-se do SVM!). Sendo assim, as implementações não estão preparadas para lidar com dados que são representados na forma de texto, como por exemplo, variáveis categóricas, sejam elas ordinais (quando existe uma **naturalmente** hierarquia do que é melhor e do que é pior) ou sejam elas nominais (não se sabe *a priori* se uma categoria é melhor ou pior que outras, pois essa inferência só pode ser feita dos dados amostrados.). Já vimos que existem muitas fontes de dados, e diferentes tipos de variáveis que podem ser extraídas dessas fontes, de forma que nós devemos ter adaptações para lidar com esses obstáculos.

<img src="https://1.bp.blogspot.com/-5Q-8Bp0BEdY/XhaKPxDvvFI/AAAAAAAAAmE/ZtURz0WKVCsBZsLhXT_TlEFar6g1CnDHgCLcBGAsYHQ/s1600/1.JPG">



Assim sendo, a técnica mais simples de transformação dos dados textuais em dados é conhecida como *Categorical Encoding*. Existem muitas outras técnicas de transformação de dados textuais, bem mais avançadas e que inclusive podem levar em consideração o sentido semântico da palavra, mas nessa aula vamos nos ater às técnicas mais simples. Em **Dados Não Estruturados**, vocês verão que existe um campo inteiramente dedicado a esse estudo, conhecido como *Natural Language Processing*.
Também é importante mencionar que nem todos os algoritmos demandam necessariamente a aplicação de codificação. Árvores de decisão e modelos em árvore, em geral, conseguem trabalhar com atributos codificados em textos.

<img src="https://mlfromscratch.com/content/images/2020/09/exercise_3-1.png" width=900px>

## 2. Label Encoding

O *Label Encoding* é a técnica mais simples de se realizar codificação de variáveis categóricas. Essencialmente funciona atribuindo simplesmente os valores numéricos para cada valor único da categoria encontrada

<img src="https://miro.medium.com/max/772/1*QQe-4476Oy3_dI1vhb3dDg.png">

No Python, implementamos o *label enconding* utilizando a classe `LabelEncoder` da biblioteca `scikit-learn`, cuja documentação pode ser encontrada [nesse link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). O `LabelEncoder` faz a identificação das categorias únicas num vetor de dados, ordena-as em ordem crescente e depois atribui os números na ordem que elas aparecem no vetor ordenado.

In [11]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, \
    FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report
from category_encoders.target_encoder import TargetEncoder

# ignorar warnings
warnings.filterwarnings('ignore')

In [12]:
# criando um vetor base
base = ['Miami', 'SaoPaulo', 'Barcelona', 'Paris']

# criando um dataframe de múltiplas amostras
categories = pd.DataFrame(np.random.choice(base, 50), columns=['city'])

# checando distribuições iniciais
categories.value_counts()

city     
Barcelona    15
Paris        13
SaoPaulo     12
Miami        10
dtype: int64

In [13]:
# aplicando o Label Encoder
le = LabelEncoder()
le.fit(categories.city)

# visualizando as classes identificadas
print('Classes Identificadas: ', le.classes_)

Classes Identificadas:  ['Barcelona' 'Miami' 'Paris' 'SaoPaulo']


In [14]:
# realizando a transformação de colunas
categories['city_le'] = le.transform(categories.city)

# checando a distribuição das classes codificadas
categories.city_le.value_counts()

0    15
2    13
3    12
1    10
Name: city_le, dtype: int64

In [15]:
# podemos querer recuperar as informações das categorias (para outros dados de teste)
city_test = pd.DataFrame(np.random.choice(list(range(4)), 20), columns=['city_cod'])

# aplicando a transformada inversa
city_test['city'] = le.inverse_transform(city_test.city_cod)

city_test.head(10)

,city_cod,city
0,1,Miami
1,1,Miami
2,0,Barcelona
3,2,Paris
4,3,SaoPaulo
5,1,Miami
6,3,SaoPaulo
7,3,SaoPaulo
8,0,Barcelona
9,3,SaoPaulo


<font color="red">No entanto, o uso do `LabelEncoder` deve ser feito com muito cuidado !!!</font>. Pensemos por um momento da implicação de simplesmente atribuir números às categorias. Fazendo dessa forma, o modelo pode receber a informação de que uma categoria é melhor que outra, simplesmente pelo fato da primeira ter recebido um valor maior que a segunda no Label Encoding. Isso é conhecido como *inferência a priori*, que significa quando o modelador inclui um conhecimento ou um viés previamente à modelagem e, portanto, o modelo terá seus parâmetros ajustados assumindo as premissas como verdadeiras. Por isso, cuidado é necessário nessa transformação.
Há casos em que a inferência *a priori* ajuda o treinamento e simplificação dos modelos. Além disso, há casos em que a simples numeração das variáveis não apresenta problema, por exemplo, quando **já existe uma hierarquia** - variáveis categóricas ordinais ("Bom", "Ruim", entre outros). Aplicar transformações de label encoding na variável alvo (problemas de classificação) também não tem problema.

Mas e como proceder nos casos de variáveis preditoras sem hierarquia prévia?

## 3. One Hot Encoding

A técnica de *One Hot Encoding* vem como uma possibilidade de solução para o problema de inferência a priori causada pela aplicação do label encoding. Essencialmente, o one-hot-encoding funciona como a transformação de uma única coluna categórica em múltiplas colunas binárias. Serão criadas tantas colunas quantas forem as classes da variável categórica.

<img src="https://images.deepai.org/glossary-terms/a609ab3c23f948f896657e4304b20ed1/onehot.jpeg">

Dessa forma, todas as variáveis categóricas são transformadas em valores binárias, que atuam como se fossem "flags" indicativas da *presença ou ausência* de cada categoria naquele registro. Dessa forma, avalia-se o efeito de forma mais justa, sem colocar viés no modelo.

In [16]:
# aplicando o One Hot Encoding no conjunto de dados das cidades
ohe = OneHotEncoder()
ohe.fit(categories[['city']])

OneHotEncoder()

In [17]:
# analisando categorias e criando novas colunas
print('Categorias analisadas:', ohe.categories_)

new_cols = [f'city_{col}' for col in ohe.categories_[0]]
print('Novas colunas: ', new_cols)

Categorias analisadas: [array(['Barcelona', 'Miami', 'Paris', 'SaoPaulo'], dtype=object)]
Novas colunas:  ['city_Barcelona', 'city_Miami', 'city_Paris', 'city_SaoPaulo']


In [18]:
# realizando transformação das features
categories[new_cols] = ohe.transform(categories[['city']]).toarray()
categories.head(10)

,city,city_le,city_Barcelona,city_Miami,city_Paris,city_SaoPaulo
0,SaoPaulo,3,0.0,0.0,0.0,1.0
1,SaoPaulo,3,0.0,0.0,0.0,1.0
2,Miami,1,0.0,1.0,0.0,0.0
3,Miami,1,0.0,1.0,0.0,0.0
4,Barcelona,0,1.0,0.0,0.0,0.0
5,Paris,2,0.0,0.0,1.0,0.0
6,Miami,1,0.0,1.0,0.0,0.0
7,Barcelona,0,1.0,0.0,0.0,0.0
8,Paris,2,0.0,0.0,1.0,0.0
9,Paris,2,0.0,0.0,1.0,0.0


Da mesma forma que o label encoding, a classe `OneHotEncoder` também pertence ao pacote `scikit-learn` e sua documentação pode ser encontrada [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

No entanto, um outro cuidado precisa ser tomado com relação ao one hot encoder. Se formos pensar na forma que as variáveis ficam dispostas, podemos notar que existe uma correlação perfeita entre todos as colunas criadas, visto que existe um grau de dependência entre as colunas. Isso quer dizer que, se conhecermos o valor de $n-1$ colunas, automaticamente sabemos o valor da outra coluna restante. Isso é chamado de *dummy variable trap* e causa a ocorrência de um viés no modelo, de forma que ele acaba superdimensionando a importância de uma dos atributos, devido à redundância de informação. Sendo assim, no momento de aplicação, deve-se eliminar uma das variáveis.

Em termos de impacto na interpretabilidade do modelo, pode-se entender que a variável que for eliminada, será o "caso-base" e que, portanto, os efeitos das outras variáveis que permanecerem podem ser interpretadas como o efeito esperando quando "se muda do caso base para a categoria selecionada".

## 4. Column Transformer e FunctionTransformer

O [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) e o [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html#sklearn.preprocessing.FunctionTransformer) são ferramentas de automação de processamento providas pelo `scikit-learn` que nos ajudam a realizar a transformação do nosso conjunto de dados.

O `FunctionTransformer` aplica uma função definida pelo usuário ou um objeto de função nativa (`np.log`), por exemplo, a uma coluna, de forma a aplicar essa função em todas as linhas do conjunto de dados.

O `ColumnTransformer` constrói um "Pipeline" de funções transformadoras de colunas, de forma a direcionar a transformação adequada para cada tipo de coluna do dataset.

Vejamos um exemplo de aplicação:

In [19]:
# carregando os dados
data = sns.load_dataset('penguins')
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [20]:
# eliminando dados nulos
data.dropna(inplace = True)

In [21]:
# separação de dados preditores e preditos
x = data.drop(['species'], axis = 1)
y = data[['species']]

In [22]:
# configurar um transformador de colunas
ct = ColumnTransformer(
    [
        ('one_hot_categ', OneHotEncoder(drop='first'), ['island', 'sex']),
        ('log_body_mass', FunctionTransformer(np.log), ['body_mass_g'])
    ],
    remainder='passthrough'
)

In [24]:
# separação do conjunto treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=19)

# criando o amostrador 
cvs = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

# criando o pipeline
pipe = Pipeline(
    [
        ('preprocessing', ct),
        ('scaler', StandardScaler()),
        ('tree', DecisionTreeClassifier())
    ]
)

# criando o dicionário de distribuições
param_dict = {
    'tree__criterion': ['entropy', 'gini'],
    'tree__max_depth': [1, 2, 3, 4, 5]
}

In [25]:
# criar e treinar o seletor de hiperparâmetros
hyp_tun = RandomizedSearchCV(pipe,
                             param_dict,
                             n_iter = 10,
                             scoring = 'accuracy',
                             refit = True,
                             random_state=19)
hyp_tun.fit(x_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('one_hot_categ',
                                                                               OneHotEncoder(drop='first'),
                                                                               ['island',
                                                                                'sex']),
                                                                              ('log_body_mass',
                                                                               FunctionTransformer(func=<ufunc 'log'>),
                                                                               ['body_mass_g'])])),
                                             ('scaler', StandardScaler()),
                                             ('tree',
                                              DecisionTreeClassifier())]),
                   param_distributions={'tree__criterion': ['entropy', 'gini'],
                                        'tree__max_depth': [1, 2, 3, 4, 5]},
                   random_state=19, scoring='accuracy')

In [26]:
# verificando melhores parâmetros e melhor métrica
print('Melhores parâmetros: ', hyp_tun.best_params_)
print('Melhor acurácia média: ', hyp_tun.best_score_)

# realizando novas predições
yhat = hyp_tun.best_estimator_.predict(x_test)

# relatório de classificação
print(classification_report(y_test, yhat))

Melhores parâmetros:  {'tree__max_depth': 5, 'tree__criterion': 'entropy'}
Melhor acurácia média:  0.9698812019566736
              precision    recall  f1-score   support

      Adelie       0.96      0.90      0.93        29
   Chinstrap       0.93      0.93      0.93        14
      Gentoo       0.92      1.00      0.96        24

    accuracy                           0.94        67
   macro avg       0.94      0.94      0.94        67
weighted avg       0.94      0.94      0.94        67



________

### Exercício 1

Agora vamos utilizar o pipeline e column-transformer num problema de regressão. Com a base 'Steel_industry_data.csv':

- carregue a base
- Elimine as seguintes colunas: `Date`
- Eliminar dados nulos
- Aplique um column transformer nas colunas WeekStatus, Day_of_Week, Load_type
- Separar 30 % para teste
- Use o Random Search com 5 iterações para selecionar os hiperparâmetros (**Cuidado** com o número de hiperparâmetros pois o conjunto de dados é grande)
- Use um objeto `KFold` para validação cruzada (use 3 folds)
- As métricas de seleção podem ser o $R^2$ ou o MAE
- Teste os modelos `LinearRegression`, e `RandomForest`
- Utilize o modelo do `Pipeline`
- Não se esqueça de aplicar o escalonamento dos dados (`StandardScaler`)

______________

## 5. Introdução ao Target Encoding

Em alguns casos, pode acontecer dos conjuntos de dados possuírem colunas com muitas classes (10 ou mais). Se formos aplicar one hot encoding para todas, cada uma dessas colunas vai produzir outras 10 ou mais colunas adicionais, de forma que o conjunto de dados final teria uma altíssima dimensionalidade. Conforme vamos ver lá em ML3, alta dimensionalidade pode ser muito ruim, de forma que o modelo pode facilmente ser *overfittado*. Assim, nesses casos, podemos utilizar uma técnica conhecida como **TargetEncoding**.

O Target Encoding é como se fosse uma codificação de variáveis categóricas de uma forma "supervisionada", onde usamos informação da variável "alvo" para codificar a variável categórica. Sendo assim, é importante realizá-la somente depois da separação treino teste para evitar o **information leakage**. Existe [esse pacote](https://contrib.scikit-learn.org/category_encoders/targetencoder.html) que implementa o Target Encoding numa forma parecida com o `scikit-learn`.

Para evitar a explosão dimensional, o Target Encoding subsitui a variável categórica pela média, mediana ou moda da variável alvo considerando somente os registros pertecentes àquela classe.